<a href="https://colab.research.google.com/github/smotrdan1/BootStrap/blob/main/144Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets --quiet
!sudo apt-get install git-lfs --quiet
!git-lfs install --quiet
!pip install wandb --quiet

     |████████████████████████████████| 4.2 MB 4.1 MB/s 
     |████████████████████████████████| 346 kB 50.7 MB/s 
     |████████████████████████████████| 596 kB 74.7 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 19.2 MB/s 
     |████████████████████████████████| 140 kB 53.6 MB/s 
     |████████████████████████████████| 1.1 MB 26.3 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 212 kB 80.0 MB/s 
     |████████████████████████████████| 127 kB 85.2 MB/s 
     |████████████████████████████████| 271 kB 68.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 10.7 MB/s 
     |████████████████████████████████| 112 kB 55.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 r

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_dataset, load_metric
import datasets

In [4]:
df = pd.read_csv('Test.csv')
df.drop(df.columns.difference(['call.business_or_person_transcript','call.business_or_person']), 1, inplace=True)
df = df[df["call.business_or_person_transcript"].str.contains("empty") == False]
df = df[df["call.business_or_person"].str.contains("NaN") == False]
df["call.business_or_person"]= df["call.business_or_person"].replace(to_replace ="פרטי", value ="private")
df["call.business_or_person"]= df["call.business_or_person"].replace(to_replace ="עסקי", value ="business")


#df.dropna(inplace=True)
#df.rename(columns={'Sentence':'text', 'Type':'label'}, inplace=True)


df.rename(columns={'call.business_or_person_transcript':'text', 'call.business_or_person':'label'}, inplace=True)
df = df.iloc[:500]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,text,label
0,אדם דתי,private
2,אדם פרטי,private
5,עסק,business
8,עסק,business
10,פרטי,private
...,...,...
808,עסק,business
809,אדם פרטי,private
812,עסק,business
813,אדם פרטי,private


In [5]:
dataset = Dataset.from_dict(df)
dataset[1] 
#descriptive_dataset = load_dataset(train_dataset = Dataset.from_dict(train_df))
#descriptive_dataset

{'label': 'private', 'text': 'אדם פרטי'}

In [6]:
def lable2ID(w):
  if w["label"] == "private":
    w["label"] = 0
  else:
    w["label"] = 1
  return w

In [8]:
descriptive_dataset = dataset.map(lable2ID)

  0%|          | 0/500 [00:00<?, ?ex/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("onlplab/alephbert-base")
# Other available Hebrew models
# tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer.tokenize("אדם פרטי", truncation=True)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

['אדם', 'פרטי']

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,padding = True)
    

In [11]:
train, validate, test = \
              np.split(df.sample(ignore_index=True, frac=1, random_state=42), 
                       [int(.8*len(df)), int(.9*len(df))])

#my_dataset_dict_train  = datasets.DatasetDict(train)
#my_dataset_dict_test  = datasets.DatasetDict(test)
#my_dataset_dict_validate  = datasets.DatasetDict(validate)
my_dataset_dict_train = (Dataset.from_dict(train)).map(lable2ID)
my_dataset_dict_test = (Dataset.from_dict(test)).map(lable2ID) 
my_dataset_dict_validate = (Dataset.from_dict(validate)).map(lable2ID)         
my_dataset_dict = datasets.DatasetDict({"train":my_dataset_dict_train ,"test":my_dataset_dict_test,"validation":my_dataset_dict_validate})

tokenized_descriptive = my_dataset_dict.map(preprocess_function, batched=True)
tokenized_descriptive

  0%|          | 0/400 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
})

In [18]:
tokenized_descriptive["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 400
})

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding=True)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='onlplab/alephbert-base', vocab_size=52000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("onlplab/alephbert-base", num_labels=2)
# Other available Hebrew models
# model = AutoModelForSequenceClassification.from_pretrained("avichr/heBERT", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Downloading:   0%|          | 0.00/481M [00:00<?, ?B/s]

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base

In [14]:
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  # print(f'perd: {predictions} {type(labels)}\nlabels: {labels} {type(labels)}')
  return metric.compute(predictions=predictions, references=labels)

# Try other metrices
# accuracy_score = load_metric('accuracy')
# f1_score = load_metric('f1')
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     # returns a dict like {'f1':0.54221}
#     f1 = f1_score.compute(predictions=predictions, references=labels)
#     # returns a dict like {'accuracy': 0.3241}
#     acc = accuracy_score.compute(predictions=predictions, references=labels)
#     # merge the two dictionaries
#     return {**f1, **acc}

In [15]:
from transformers import TrainingArguments, Trainer
path = './finetuning_results'
training_args = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # label_names = ['Descriptive', 'NotDescriptive'],
    report_to="wandb",
    logging_steps=48,
    evaluation_strategy="epoch",
    num_train_epochs=12,
    weight_decay=0.01,
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_descriptive["train"],
    eval_dataset=tokenized_descriptive["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [17]:
trainer.train()
trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 300
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.023124,1.000000
2,0.088800,0.013396,1.000000
3,0.088800,0.010439,1.000000
4,0.009400,0.008951,1.000000
5,0.009400,0.007276,1.000000
6,0.000700,0.007170,1.000000
7,0.000700,0.006951,1.000000
8,0.000600,0.006903,1.000000
9,0.000600,0.006642,1.000000
10,0.000300,0.006574,1.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 16
The following colum

{'epoch': 12.0,
 'eval_accuracy': 1.0,
 'eval_loss': 0.006490940228104591,
 'eval_runtime': 1.5133,
 'eval_samples_per_second': 33.041,
 'eval_steps_per_second': 2.643}

In [27]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machi

In [50]:
git_path = "DanielSM/144Test"
trainer.push_to_hub("DanielSM/144Test")

Saving model checkpoint to ./finetuning_results
Configuration saved in ./finetuning_results/config.json
Model weights saved in ./finetuning_results/pytorch_model.bin
tokenizer config file saved in ./finetuning_results/tokenizer_config.json
Special tokens file saved in ./finetuning_results/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 1.0}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/DanielSM/finetuning_results
   4f85a88..30a8249  main -> main



In [29]:
tokenized_descriptive

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
})

In [30]:
dataset = tokenized_descriptive

In [31]:
def labeltoId(s):
  if s == "Descriptive":
    return 0
  else: 
    return 1

In [51]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("DanielSM/finetuning_results")

model = AutoModelForSequenceClassification.from_pretrained("DanielSM/finetuning_results")

https://huggingface.co/DanielSM/finetuning_results/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp8pfxnn68


Downloading:   0%|          | 0.00/365 [00:00<?, ?B/s]

storing https://huggingface.co/DanielSM/finetuning_results/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1e3e9eb42e2437b358760d41f89d1e8187cd493c88e5faa63fc93cdcd04adfed.46460c154c02c720065549d1deee7f214616a22367001427c9d9e14f132906bc
creating metadata file for /root/.cache/huggingface/transformers/1e3e9eb42e2437b358760d41f89d1e8187cd493c88e5faa63fc93cdcd04adfed.46460c154c02c720065549d1deee7f214616a22367001427c9d9e14f132906bc
https://huggingface.co/DanielSM/finetuning_results/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp5kpa70__


Downloading:   0%|          | 0.00/532k [00:00<?, ?B/s]

storing https://huggingface.co/DanielSM/finetuning_results/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/b72f892420a5bd7a5facf668ac2dedbf402e4e47ac11763762fd125fb6327ac2.68d0ec0be97fb095ce7b008c055f41d9e086d78cef089bacc9e3d6c4ac9957c1
creating metadata file for /root/.cache/huggingface/transformers/b72f892420a5bd7a5facf668ac2dedbf402e4e47ac11763762fd125fb6327ac2.68d0ec0be97fb095ce7b008c055f41d9e086d78cef089bacc9e3d6c4ac9957c1
https://huggingface.co/DanielSM/finetuning_results/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpve48ylo0


...

storing https://huggingface.co/DanielSM/finetuning_results/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/827d02ee806a6622d55639bf5e4c7e9c7db83e49af6191ce61a25c690c4e3f15.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/827d02ee806a6622d55639bf5e4c7e9c7db83e49af6191ce61a25c690c4e3f15.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/DanielSM/finetuning_results/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/b72f892420a5bd7a5facf668ac2dedbf402e4e47ac11763762fd125fb6327ac2.68d0ec0be97fb095ce7b008c055f41d9e086d78cef089bacc9e3d6c4ac9957c1
loading file https://huggingface.co/DanielSM/finetuning_results/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/e815591586ff6a1f50bd159cc44529b887a8dd9e9ef32e8fad4a75ffda161c9d.2766bd42370afcd704b18cd8ea01da072aeb5dc94a6c08e8f25

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

storing https://huggingface.co/DanielSM/finetuning_results/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/f2dbffe7be20637461fe478c6ac431e50a9801b0239c6ba96e3cb25c100e3584.19378c6720467a9ae6dbdb4dde83f12d8ac839a9eb4fe44fca08690d6f68b670
creating metadata file for /root/.cache/huggingface/transformers/f2dbffe7be20637461fe478c6ac431e50a9801b0239c6ba96e3cb25c100e3584.19378c6720467a9ae6dbdb4dde83f12d8ac839a9eb4fe44fca08690d6f68b670
loading configuration file https://huggingface.co/DanielSM/finetuning_results/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f2dbffe7be20637461fe478c6ac431e50a9801b0239c6ba96e3cb25c100e3584.19378c6720467a9ae6dbdb4dde83f12d8ac839a9eb4fe44fca08690d6f68b670
Model config BertConfig {
  "_name_or_path": "DanielSM/finetuning_results",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act"

Downloading:   0%|          | 0.00/481M [00:00<?, ?B/s]

storing https://huggingface.co/DanielSM/finetuning_results/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/68f4585f6b80826edec9047f1ad1afd07ff8ab1e42da80fe034da4451b20c32a.6bcf528a679b5909f766ca8da1a85480fd262bf029be548706bdf9f19e2e0ad7
creating metadata file for /root/.cache/huggingface/transformers/68f4585f6b80826edec9047f1ad1afd07ff8ab1e42da80fe034da4451b20c32a.6bcf528a679b5909f766ca8da1a85480fd262bf029be548706bdf9f19e2e0ad7
loading weights file https://huggingface.co/DanielSM/finetuning_results/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/68f4585f6b80826edec9047f1ad1afd07ff8ab1e42da80fe034da4451b20c32a.6bcf528a679b5909f766ca8da1a85480fd262bf029be548706bdf9f19e2e0ad7
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at DanielSM/finetuning_results.
If your task is similar to the task t

In [36]:
acc_metric = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

In [52]:
references_lst = []
predictions_lst = []
for batch in dataset["test"]:
  inputs = batch["text"]
  reference =  labeltoId(batch["label"])
  logits = model(**tokenizer(inputs, return_tensors='pt',truncation=True,padding=True))
  prediction = np.argmax(logits['logits'].detach().numpy(), axis=1)
  references_lst.append(reference)
  predictions_lst.append(prediction)
f1 = f1_metric.compute(predictions=predictions_lst, references=references_lst)
acc = acc_metric.compute(predictions=np.array(predictions_lst), references=np.array(references_lst))
  # merge the two dictionaries
print(f1, acc)

{'f1': 0.8505747126436781} {'accuracy': 0.74}


In [59]:
from transformers import pipeline
classifier = pipeline("text-classification",model='DanielSM/finetuning_results')
output = classifier("קופת חולים לאומית")
output

loading configuration file https://huggingface.co/DanielSM/finetuning_results/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f2dbffe7be20637461fe478c6ac431e50a9801b0239c6ba96e3cb25c100e3584.19378c6720467a9ae6dbdb4dde83f12d8ac839a9eb4fe44fca08690d6f68b670
Model config BertConfig {
  "_name_or_path": "DanielSM/finetuning_results",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,

[{'label': 'LABEL_1', 'score': 0.7308353185653687}]